<a href="https://colab.research.google.com/github/mjooee/Lietector-/blob/models/chatbot_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SentenceTransformer를 사용한 Fine-Tuning

사용한 사전학습된 모델: jhgan/ko-sroberta-multitask

사용한 데이터셋: 대처방법 챗봇 qa_중복질문쌍

In [ ]:
!pip install sentence-transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request

In [ ]:
from sentence_transformers import SentenceTransformer, models
from datasets import Dataset

#모델 준비
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

#데이터 준비
train_data = pd.read_excel('/content/drive/MyDrive/대처방법_챗봇.xlsx', sheet_name = '중복질문쌍')
train_data.head()


,train
0,보이스피싱 피해 신고는 어디에 해야하나요?
1,보이스피싱으로 의심되는 경우 경찰에 신고 하면 되나요?
2,보이스피싱 신고하고 싶습니다.
3,보이스피싱 피해는 어디로 신고해야 하나요?
4,어디에 보이스피싱을 신고할 수 있나요?


In [ ]:
train_data.iloc[0:5]

,train
0,보이스피싱 피해 신고는 어디에 해야하나요?
1,보이스피싱으로 의심되는 경우 경찰에 신고 하면 되나요?
2,보이스피싱 신고하고 싶습니다.
3,보이스피싱 피해는 어디로 신고해야 하나요?
4,어디에 보이스피싱을 신고할 수 있나요?


In [ ]:
def make_dataset(data_list):
    dataset = []
    for i in range(0, len(data_list), 5): #같은 set으로 묶일 데이터 수
        small_dict = dict(set = data_list[i : i+5])
        dataset.append(small_dict)
    return dataset

In [ ]:
import math

#딕셔너리로 구성된 데이터 리스트 만들기
train_list = train_data['train'].values.tolist()
#test_list = train_data['test'].values.tolist() #데이터 프레임의 데이터 모두 리스트로 저장
#test_list = test_list[0:train_data['test'].count()] #nan 제거

In [ ]:
#각각 리스트로 저장
train_dataset = make_dataset(train_list)
#test_dataset = make_dataset(test_list)

In [ ]:
train_dataset = Dataset.from_list(train_dataset)
#test_dataset = Dataset.from_list(test_dataset)

In [ ]:
#train과 test 묶어서 사전으로 저장(예시 데이터 셋을 그대로 따라한 거라 왜 사전으로 굳이 묶는지는 몰라)
dataset = {"train": train_dataset} #"test" : test_dataset
dataset

{'train': Dataset({
     features: ['set'],
     num_rows: 17
 })}

In [ ]:
from sentence_transformers import InputExample

train_examples = []
train_data = dataset['train']['set']
n_examples = dataset['train'].num_rows

for i in range(n_examples):
  example = train_data[i]
  train_examples.append(InputExample(texts=[example[0], example[1]]))

In [ ]:
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=64)#원래 값 64
train_loss = losses.MultipleNegativesRankingLoss(model=model)
num_epochs = 10
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
model.save(path = "/content/drive/MyDrive/chatbot_5", model_name = "chatbot_embedding_model")